In [1]:
# Instalar e Importar librerias para dar lectura a los datos y mostrar cada dataframe
# !pip install pandas
import pandas as pd
import json

In [2]:
#-------------Conexión necesaria con Google Drive para manipular las bases de datos-------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
#---------------- Lectura de datos y Recoleccion de ellos ----------------
# Leemos todos los datos
DFFB = pd.read_json('/content/drive/MyDrive/Proyecto/facebook_data.json', lines=False)
DFIN = pd.read_json('/content/drive/MyDrive/Proyecto/instagram_data.json', lines=False)
DFX = pd.read_json('/content/drive/MyDrive/Proyecto/twitter_data.json', lines=False)

In [15]:
import re

In [22]:
# Preprocesamiento de datos Tokenización, Lematización, Eliminación de StopWords y Normalización de textos
# ---------------------Usando la base de datos de Facebook--------------------
# Importar Bibliotecas Necesarias
import pandas as pd
import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#-----------------Recursos necesarios para aplicar el preprocesamiento en español del texto-----------------
# Descargar recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# Cargar el modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

#------------Aplicar una limpieza de los datos-----------------
#Crear una funcion para limpiar datos

def limpiar_datos(texto):
  """
  Normalizacion del texto: conversión minusculas y eliminar carcateres especiales.
  """
  # Eliminar caracteres especiales y conversión a minúsculas
  texto = texto.lower()                                    #Convierto a minúsculas
  texto = re.sub(r'[^a-zA-ZáéíóúñüÁÉÍÓÚÑÜ\s]', '', texto)  #Elimino caracteres especiales
  texto = re.sub(r'[^\w\s+]', ' ', texto)                  #Elimino signos de puntuación
  texto = re.sub(r'\d+', ' ', texto)                       #Elimino números
  # texto = re.sub(r'\s+', ' ', texto)                     #Elimino espacios en blanco
  texto = texto.strip()                                    #Elimino espacios en blanco al inicio y al final del texto extras
  return texto

#---------Aplicación del Preprocesamiento de datos-----------------

def preprocesamiento_datos(texto):
  """
  Función para aplicar el preprocesamiento de datos, tokenización, lematización, eliminación de stopwords y normalización de textos.
  """
  # Aplicar la función de limpieza de datos
  texto = limpiar_datos(texto)

  # Tokenización
  tokens = word_tokenize(texto, language='spanish')

  # Eliminar stopwords en español
  stop_words = set(stopwords.words('spanish'))
  tokens_sin_stop = [word for word in tokens if word not in stop_words]

  # Lematización con spaCy
  doc = nlp(' '.join(tokens_sin_stop))
  tokens_lematizados = [token.lemma_ for token in doc]

  # Unir los tokens procesados
  texto_procesado = ' '.join(tokens_lematizados)

  return texto_procesado

#----------Pruebas con textos en español--------
# texto = "¡Los niños juegan en el parque  mientras corrían y estaban saltando felices!"
texto = DFFB['contenido'].to_string()

# Aplicar procesamiento
DFFB['Texto_Procesado'] = DFFB['contenido'].apply(preprocesamiento_datos)
# print("Texto original: \n", texto)
# print("Texto procesado: ", texto_procesado)

# print(FB['Texto_Procesado'],FB['contenido'])
# print(FB['Texto_Procesado'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [23]:
DFFB

,id,usuario,fecha,contenido,likes,comentarios,compartidos,Texto_Procesado
0,1,usuario1,2025-09-19,"Una experiencia maravillosa, muy satisfecho. #...",79,87,26,experiencia maravilloso satisfecho satisfaccion
1,2,usuario2,2025-09-05,El peor servicio que he recibido. #colera,341,98,7,peor servicio recibido colera
2,3,usuario3,2025-03-13,"Este servicio es excelente, estoy muy satisfec...",144,25,41,servicio excelente satisfecho satisfaccion
3,4,usuario4,2025-07-28,No puedo creer lo mal que funciona esto. #colera,366,41,37,poder creer mal función colera
4,5,usuario5,2025-05-14,"Esto es increible, estoy extasiado! #extasis",451,4,31,increible extasiado extasis
...,...,...,...,...,...,...,...,...
895,896,usuario896,2025-01-22,"Esto es increible, estoy extasiado! #extasis",431,42,10,increible extasiado extasis
896,897,usuario897,2025-04-21,"Esto es increible, estoy extasiado! #extasis",466,89,0,increible extasiado extasis
897,898,usuario898,2025-10-29,Es frustrante que no funcione como esperaba. #...,332,28,6,frustrante funcione esperar frustracion
898,899,usuario899,2025-08-30,"No volvere a comprar aqui, muy decepcionado. #...",260,89,17,volverir comprar aqui decepcionado decepcion


In [25]:
!pip install deep_translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00


In [26]:
#---------------- Obtener sentimiento dependiendo del comentario en cada DF------------------
from textblob import TextBlob
from deep_translator import GoogleTranslator

# Función para obtener el sentimiento
def obtener_sentimiento(texto):
    try:
        texto_traducido = GoogleTranslator(source='auto', target='en').translate(texto)
        polaridad = TextBlob(texto_traducido).sentiment.polarity
    except:
        # polaridad = TextBlob(texto).sentiment.polarity
        polaridad = 0
        # return 'Neutral'

    if polaridad > 0:
        return 'Positivo'
    elif polaridad == 0:
        return 'Positivo'
    else:
        return 'Negativo'

# Aplicar la función a la columna 'Texto_Procesado' y crear una nueva columna 'Sentimiento'
# DFFB.drop('Sentimiento', axis=1, inplace=True)
DFFB['Sentimiento'] = DFFB['contenido'].apply(obtener_sentimiento)
DFFB2 = DFFB[['Texto_Procesado','Sentimiento']]

In [27]:
DFFB2

,Texto_Procesado,Sentimiento
0,experiencia maravilloso satisfecho satisfaccion,Positivo
1,peor servicio recibido colera,Negativo
2,servicio excelente satisfecho satisfaccion,Positivo
3,poder creer mal función colera,Negativo
4,increible extasiado extasis,Positivo
...,...,...
895,increible extasiado extasis,Positivo
896,increible extasiado extasis,Positivo
897,frustrante funcione esperar frustracion,Negativo
898,volverir comprar aqui decepcionado decepcion,Negativo


In [28]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Cargar el modelo en español
nlp = spacy.load('es_core_news_sm')

# Textos de ejemplo y sus etiquetas de sentimiento
textos = DFFB2['Texto_Procesado'].astype(str).tolist()
sentimientos = DFFB2['Sentimiento'].astype(str).tolist()

# Preprocesar los textos
def preprocesar_texto(texto):
    doc = nlp(texto)
    return " ".join([token.lemma_ for token in doc if not token.is_stop])

textos_procesados = [preprocesar_texto(texto) for texto in textos]

# Vectorizar los textos
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(textos_procesados)
y = sentimientos

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar un modelo SVM
modelo = SVC(kernel='linear')
modelo.fit(X_train, y_train)

# Evaluar el modelo
y_pred = modelo.predict(X_test)
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))

Reporte de clasificación:
               precision    recall  f1-score   support

    Negativo       1.00      1.00      1.00       132
    Positivo       1.00      1.00      1.00       138

    accuracy                           1.00       270
   macro avg       1.00      1.00      1.00       270
weighted avg       1.00      1.00      1.00       270

Precisión: 1.0
